<h2 style= "text-align:center"> <b>Đồ án </b> </h2>
<h2 style='text-align:center;font-size:32px'>Lập trình song song</h2>

 <font color='289C4E'>NỘI DUNG<font><a class='anchor' id='top'></a>
    
[1. Mô tả đồ án](#1)
        
[2. Cài đặt tuần tự](#2)
        
[3. Cài đặt song song](#3)

[4. Cài đặt song song và tối ưu](#4)

[5. Tự đánh giá đồ án](#5)

[6. Phân công](#6)
    
[7. Tài liệu tham khảo](#7)
    

**Thành viên nhóm:**
- 20120481 - Phan Xuân Hoài
- 20120534 - Nguyễn Minh Nghĩa

<a id="1"></a>
<div style=" background-color:#cdf7c1 ; color:#e3395e; padding: 15px; border-radius:5px;text-align: center; border-style: solid; border-color: green; font-size: 35px">1. Mô tả đồ án </div>

<a id="1_1"></a>
<h3 style='color:#6ac61f; font-size: 25px'> a. Giới thiệu ứng dụng</h3>

- Trong đồ án này, nhóm em thực hiện cài đặt song song và tối ưu hóa trong giai đoạn forward-pass cho lớp Convolution trong mạng CNN (LetNet5) sử dụng CUDA. Với lớp Convolution là các lớp chính của mạng nơ-ron tích chập (CNN), được sử dụng trong nhiều nhiệm vụ học máy các như phân loại hình ảnh, phát hiện đối tượng, xử lý ngôn ngữ tự nhiên.

![Kiến trúc Lenet-5](./image/lenet-5.jpeg "Kiến trúc Lenet-5")

    Kiến trúc Lenet5 sử dụng trong đồ án:
        Layer 1: 6 kernel với kích thước 5 × 5, sử dụng hàm ReLU activation.
        Layer 2: Max Pooling với kích thước 2 × 2
        Layer 3: 16 kernel với kích thước 5 × 5,sử dụng hàm ReLU activation.
        Layer 4: Max Pooling với kích thước 2 × 2
        Layer 5: dense layer với 120 outputs, sử dụng hàm ReLU activation
        Layer 6: dense layer với 84 outputs, sử dụng hàm ReLU activation
        Layer 7: dense layer với 10 outputs, sử dụng hàm ReLU activation

- Nhóm em sử dụng framework [mini-dnn-cpp (Mini-DNN)](https://github.com/iamhankai/mini-dnn-cpp/tree/master) làm điểm khởi đầu để triển khai phiên bản tuần tự và song song hóa lớp Convolution của LeNet-5.

- Bộ dữ liệu Fashion MNIST sẽ được sử dụng để huấn luyện và kiểm tra, gồm các hình ảnh 1 chiều kích thước 28 x 28 pixel bao gồm 10 lớp đầu ra (áo phông, váy, giày thể thao, bốt, v.v.).


<a id="1_2"></a>
<h3 style='color:#6ac61f; font-size: 25px'>  b. Động lực thực hiện</h3>

**Nâng cao tốc độ chương trình:** Quá trình song song hóa cho lớp Convolution của Lenet-5 với CUDA giúp cải thiện tốc độ thực thi của chương trình để phù hợp với thời gian thực.

**Nâng cao kiến thức và kỹ năng:** Quá trình song song hóa cho lớp Convolution của Lenet-5 với CUDA giúp nhóm em nắm vững hơn lý thuyết và thực hành về tính toán song song, cài đặt mô hình với ngôn ngữ c++, cuda.



<a id="1_3"></a>
<h3 style='color:#6ac61f; font-size: 25px'> c. Tổng quan đóng góp</h3>

| MSSV     | Họ tên  | Phần trăm đóng góp|
| -------- | ------- |-------------------|
| 20120481  | Phan Xuân Hoài   |     45%    |
| 20120534 | Nguyễn Minh Nghĩa    |   55%   |


<a id="2"></a>
<div style=" background-color:#cdf7c1 ; color:#e3395e; padding: 15px; border-radius:5px;text-align: center; border-style: solid; border-color: green; font-size: 35px">2. Cài đặt tuần tự </div>

<a id="2_1"></a>
<h3 style='color:#6ac61f; font-size: 25px'>a. Flow thiết kế</h3>

- Chỉnh sửa lại kiến trúc lớp Framework Mini-DNN thành Lenet-5 và train trên tập dữ liệu Fashion MNIST và lưu lại tham số.
- Thiết lập mạng CNN Lenet-5 tuần tự và load tham số vào mô hình.
- Với mỗi ảnh input đưa vào mô hình sẽ được xử lý tuần tự và cho ra kết quả là 1 trong 10 lớp như hình:

![Flow-Sequential](./image/sequential_flow.jpg "Flow tuần tự")

**Trong các lớp Convolution, hàm foward sẽ thực hiện:**

 ![im2col](./image/im2col.png)
 
 <p style="text-align: center;">Nguồn: <a href="https://leonardoaraujosantos.gitbook.io/artificial-inteligence/machine_learning/deep_learning/convolution_layer/making_faster">Making faster</a></p>

 + Filter sẽ thực hiện dịch chuyển trên ma trận ảnh đầu vào và chuyển dữ liệu thành dạng cột tạo thành ma trận có kích thước `(height_out x width_out, height_filter x width_filter x channel_in)`.
 + Filter cũng sẽ được chuyển thành dạng ma trận dạng cột với kích thước `(channel_in x height_filter x width_filter, channel_out)`.

 + Thực hiện nhân hai ma trận trên và cộng với bias ta được ma trận đầu ra: `(height_out x width_out, channel_out)`.


<a id="2_2"></a>
<h3 style='color:#6ac61f; font-size: 25px'> b. Đánh giá</h3>

### Mô tả flow thực nghiệm:
+ Tải model vào mạng CNN
+ Bắt đầu đo thời gian cho lớp Convolution 1
+ Thực hiện vòng lặp qua cho mỗi ảnh
+ Thực hiện tính toán tích chập cho mỗi ảnh
+ Kết thúc đo thời gian
+ Xuất kết quả thời gian chạy và độ chính xác
+ Tương tự cho lớp Convolution 3

### Thời gian thực thi
Được tính dựa trên thời gian tính tích chập cho toàn bộ ảnh dữ liệu đầu vào ở các lớp Convolution 1 và Convolution 3.

### Tính đúng đắn
Độ chính xác khi train là 82.96%.


### Kết quả thực nghiệm:
![Thời gian host thực thi](./image/host.png "Thời gian và độ chính xác của host")

> **Nhận xét:** Mặc dù độ chính xác đạt 82.96% nhưng thời gian thực thi tuần tự trên host rất chậm, cần song song hóa để tăng tốc độ.

<a id="3"></a>
<div style=" background-color:#cdf7c1 ; color:#e3395e; padding: 15px; border-radius:5px;text-align: center; border-style: solid; border-color: green; font-size: 35px">3. Cài đặt song song hóa </div>

<a id="3_1"></a>
<h3 style='color:#6ac61f; font-size: 25px'> a. Phân tích</h3>

**Bước có thể song song hóa:** di chuyển ma trận filter trên ma trận input đầu vào và tính tích chấp cho các phần tử tương ứng trong hai lớp convolution.

**Lý do:** lớp Convolution đóng vai trò quan trọng trong việc xử lý hình ảnh tuy nhiên lớp này tốn nhiều thời gian cho việc di chuyển ma trận filter trên ma trận input đầu vào tính tích chấp cho các phần tử tương ứng. Ta có thể thực hiện song song hóa việc này để tăng tốc độ cho lớp Convolution.

<a id="3_2"></a>
<h3 style='color:#6ac61f; font-size: 25px'>b. Thiết kế</h3>

### Flow thiết kế
- Khi hàm foward của lớp Convolution được gọi: hàm gọi kernel sẽ được thực thi bao gồm các nhiệm như cấp phát bộ nhớ, copy dữ liệu, gọi hàm kernel.
- Block có kích thước là (BLOCKSIZE, BLOCKSIZE, 1). Với BLOCKSIZE =  16.
- Grid có kích thước như sau:

![Kernel Size](./image/kernel_size.jpg "Kernel Size")

   > Trong đó:
   > + height_out, weight_out: là kích thước đầu ra của ảnh
   > + out_channel: là số lượng kênh đầu ra
   > + n_samples: là số lượng mẫu
- Khi đó mỗi thread sẽ xử lý một pixel output (của 1 output channel) bằng cách tính tích chập giữa các pixel tương ứng trên tất cả kênh của input và ma trận filter.

![Xử lý ở mỗi thread trong Convolution 1](./image/basic_conv1.jpg "Xử lý ở mỗi thread trong Convolution 1")

### Mô tả kỹ thuật
Với kích thước của Gird là
$$(n\_samples, out\_channel, \frac{height\_out * width\_out - 1}{BLOCKSIZE * BLOCKSIZE} + 1)$$


và kích thước Block là
$$ (BLOCKSIZE, BLOCKSIZE, 1) $$

> Mỗi thread sẽ xử lý cho 1 pixel ouput

- Trong mỗi thread:
    +  Tính index row, col của pixel đầu ra trong ảnh đầu ra.
    +  Tính index channel, index sample mà pixel trên thuộc vào.
    +  Lặp theo chiều dọc, ngang của filter và số kênh đầu vào:
        + Tính tích chập giữa từng phần tử của filter và phần tử ảnh đầu vào.
    + Kết quả cuối cùng là giá trị pixel trong ouput tại vị trí được tính dựa vào row, col, index channel, index sample đã tính.

```
conv_basic_kernel(output, input, weight, bias)
{
    // Tính index của thread hiện tại
    index_per_block ;

    r =  row of the output image matrix
    c =  col of the output image matrix

    IF r, c is in output matrix
        FOR EACH element of filter row         
            FOR EACH element of filter column
               FOR EACH index of input_channel      
                    sum += input[input_index] * weight;
                

        output[out_index] = sum + bias;
}
```

<a id="3_2"></a>
<h3 style='color:#6ac61f; font-size: 25px'>c. Đánh giá</h3>

### Mô tả flow thực nghiệm

1. Tải model vào mạng CNN
2. Khi hàm forward của lớp Convolution được gọi bắt đầu đo thời gian thực thi
3. Lấy các giá trị weight, bias, input
4. Gọi hàm thực hiện cấp phát bộ nhớ trên GPU và copy dữ liệu vào GPU.
5. Gọi hàm thực hiện kernel và đo thời gian thực thi kernel
6. Kết thúc đo thời gian.
7. Xuất kết quả thời gian chạy và độ chính xác


### Tốc độ thực thi

Tốc độ thực thi đươc đo bằng cách tạo Event trong cuda và ghi lại thời điểm bắt đầu và kết thúc cuối cùng sử dụng hàm cudaEventElapsedTime để tính khoảng thời gian giữa hai thời điểm.

![Thời gian thực thi](./image/time_basic.jpg "Thời gian thực thi")

| | C1-kernel | C1-Conv | C3-Kernel| C3-Conv|
|--- |----|-----|--------|---------|
|Host|    |34143.6 |     | 30042.6 |         |
|Basic|122.58|125.528|61.1396|65.5313|

**Nhận xét:** Thời gian thực thi trên hơi lớp convolution sau khi được song song hóa đã được tăng tốc lên rất nhiều. Với lớp Convolution 1 là gấp khoảng 272 lần, Convolution 3 là gấp 458 lần.

### Tính đúng đắn

![accuracy](./image/accuracy.png)

Độ chính xác đạt 82.96% do đó kết quả khi chạy song song hóa cho lớp Convolution là chính xác vì kết quả này tương đương với khi thực thi trên host.

<a id="4"></a>
<div style=" background-color:#cdf7c1 ; color:#e3395e; padding: 15px; border-radius:5px;text-align: center; border-style: solid; border-color: green; font-size: 35px">4. Cài đặt song song hóa và tối ưu</div>

<a id="4_1"></a>
<h3 style='color:#6ac61f; font-size: 25px'>  a. Loop unrolling</h3>

### Bước có thể tối ưu

Trong kernel, mỗi thread sẽ phải thực hiện ba vòng lặp lồng nhau gồm lặp theo cột, theo hàng, theo số kênh đầu vào. Điều này làm cho thời gian thực thi của kernel tăng. Ta có thể phá bỏ vòng lặp, để giảm thời gian.


### Thiết kế

Sử dụng chỉ thị `#pragma unroll` của CUDA trước vòng lặp để yêu cầu trình biên dịch thực hiện việc unroll vòng lặp, tức là thực hiện nhiều hơn trong cùng một câu lệnh hơn trong vòng lặp để tối ưu hóa hiệu suất chương trình. Tuy nhiên, #pragma unroll cũng có thể làm giảm hiệu suất chương trình trong một số trường.

### Đánh giá

- Mô tả flow thực nghiệm: tương tự với song song hóa cơ bản.
- Thời gian thực thi: Được tính tương tự với song song hóa cơ bản


![unroll](./image/unroll.png)




| | C1-kernel | C1-Conv | C3-Kernel| C3-Conv|
|--- |----|-----|--------|---------|
|Host|    |34143.6 |     | 30042.6 |         |
|Basic|122.58|125.528|61.1396|65.5313|
|Unroll|120.787|124.16|51.8714|56.0254|

> **Nhận xét:** Loop unroll giúp giảm thời gian thực thi của kernel tuy nhiên không đáng kể với Convolution 1 và khoảng 10 ms đối với lớp Convolution 3.

- Tính đúng đắn:

![accuracy](./image/accuracy.png)


Độ chính xác đạt 82.96% do đó kết quả khi chạy song song hóa với tối ưu loop unroll cho lớp Convolution là chính xác vì kết quả này tương đương với khi thực thi trên host.


<a id="4_2"></a>
<h3 style='color:#6ac61f; font-size: 25px'>  b. Sử dụng shared memory và constant memory</h3>

### Bước có thế tối ưu

- Dữ liệu ảnh input đầu vào được các thread đọc nhiều lần do đó có thể sử dụng shared memory để tăng tốc độ đọc.
- Filter, bias được đọc nhiều lần và không thay đổi do đó có thể lưu vào constant memory để tăng tốc độ cho kernel.


### Flow thiết kế

Trong mỗi thread đọc điểm ảnh từ global memory vào shared memory (1  thread có thể đọc nhiểu điểm ảnh)


### Mô tả kỹ thuật
- Shared memory là một loại bộ nhớ rất nhanh và được sử dụng để tối ưu hóa việc truy cập dữ liệu giữa các luồng trong cùng một block. Dữ liệu trong bộ nhớ này có thể được truy cập và cập nhật bởi tất cả các luồng trong block một cách nhanh chóng, giúp tăng hiệu suất tính toán song song.
- Constant memory dùng để lưu trữ dữ liệu mà các luồng có thể truy cập nhưng không thể thay đổi trong quá trình thực thi. Nó thường nhanh và có thể cung cấp hiệu suất tốt khi các luồng cần truy cập cùng một giá trị không đổi.


### Đánh giá
- Mô tả flow thực nghiệm: tương tự với song song hóa cơ bản.
- Thời gian thực thi: Được tính tương tự với song song hóa cơ bản

![image.png](./image/smem_cmem.png)

| | C1-kernel | C1-Conv | C3-Kernel| C3-Conv|
|--- |----|-----|--------|---------|
|Host|    |34143.6 |     | 30042.6 |         |
|Basic|122.58|125.528|61.1396|65.5313|
|Unroll|120.787|124.16|51.8714|56.0254|
|smem+cmem| 120.955 | 124.2 |47.0913|50.4288|

> **Nhận xét:** Sử dụng shared memory và constant memory giúp giảm thời gian thực thi của kernel so với khi chưa tối ưu tuy nhiên không đáng kể với Convolution 1 và khoảng 14 ms đối với lớp Convolution 3.

- Tính đúng đắn:

Độ chính xác đạt 82.96% do đó kết quả khi chạy song song hóa với tối ưu sử dụng shared memory và constant memory cho lớp Convolution là chính xác vì kết quả này tương đương với khi thực thi trên host.


<a id="4_3"></a>
<h3 style='color:#6ac61f; font-size: 25px'> c. Sử dụng stream</h3>

### Bước có thế tối ưu

- Các phần tử của dữ liệu input và ouput sử dụng trong kernel không có tác động lẫn nhau do đó ta có thể chia nhỏ các input, output và đưa nó vào stream để có thể overlap nhau, giảm thời gian phải chờ đợi việc copy dữ liệu từ host vào device và ngược lại.


### Flow thiết kế

- Tạo ra số lượng stream muốn sử dụng
- Chia input và ouput của kernel thành các khoảng tương ứng với số stream
- Với mỗi stream gọi kernel và truyền vào phần input, ouput tương ứng


### Mô tả kỹ thuật

Một stream trong CUDA là một chuỗi tuần tự các tác vụ tính toán hoặc truyền dữ liệu mà có thể được thực hiện đồng thời trên GPU. Bằng cách sử dụng các stream khác nhau, ta có thể chạy các tác vụ song song trên cùng một GPU.


### Đánh giá
- Mô tả flow thực nghiệm: tương tự với song song hóa cơ bản.
- Thời gian thực thi: Được tính tương tự với song song hóa cơ bản

![stream_time](./image/stream.png "Thời gian thực thi với stream")



| | C1-kernel | C1-Conv | C3-Kernel| C3-Conv|Accuracy|
|-|-----------|---------|----------|---------|---------|
|Host|    |34143.6 |     | 30042.6 |     0.8269 |
|Basic|122.58|125.528|61.1396|65.5313|0.8269|
|Unroll|120.787|124.16|51.8714|56.0254|0.8269|
|smem+cmem| 120.955 | 124.2 |47.0913|50.4288|0.8269|
|stream|13.2949|18.179|15.188|18.6921|0.8269|

> **Nhận xét:** Sử dụng stream giúp giảm đáng kể thời gian thực thi của kernel so với khi chưa tối ưu. Với Convolution 1 chạy nhanh gấp 9 lần và khoảng 4 lần đối với lớp Convolution 3.

- Tính đúng đắn:

![accuracy](./image/accuracy.png)

Độ chính xác đạt 82.96% do đó kết quả khi chạy song song hóa với tối ưu sử dụng stream cho lớp Convolution là chính xác vì kết quả này tương đương với khi thực thi trên host.


<a id="4_4"></a>
<h3 style='color:#6ac61f; font-size: 25px'> d. Kết hợp tất cả các phương pháp ở trên</h3>

### Bước có thế tối ưu

- Các phương pháp đã sử dụng ở trên có thể kết hợp với nhau


### Mô tả kỹ thuật
- Sử dụng Loop Unroll
- Sử dụng shared memory và constant memory
- Sử dụng stream

### Đánh giá
- Mô tả flow thực nghiệm: tương tự với song song hóa cơ bản.
- Thời gian thực thi: Được tính tương tự với song song hóa cơ bản

![image-4.png](./image/mix.png)

| | C1-kernel | C1-Conv | C3-Kernel| C3-Conv|Accuracy|
|--- |----|-----|--------|---------|----------|
|Host|    |34143.6 |     | 30042.6 | 0.8269        |
|Basic|122.58|125.528|61.1396|65.5313|0.8269|
|Unroll|120.787|124.16|51.8714|56.0254|0.8269|
|smem+cmem| 120.955 | 124.2 |47.0913|50.4288|0.8269|
|Stream|13.2949|18.179|15.188|18.6921|0.8269|
|All|13.3483|16.6544|9.99354|13.2731|0.8269|

> **Nhận xét:** Sử dụng stream giúp giảm đáng kể thời gian thực thi của kernel so với khi chưa tối ưu. Tuy nhiên so với chỉ sử dụng stream thì hiệu quả không có quá nhiều khác biệt khi gần như tương đương ở Convolution 1 và nhanh hơn khoảng 5 ms ở Convolution 3.




**Nhận xét chung:**

    + Mặc dù loop unroll và sử dụng shared memory, constant memory có mang lại hiệu quả tuy nhiên không quá lớn và chủ yếu ở Convolution 3.
    
    + So với 2 phương pháp loop unroll và sử dụng shared memory, constant memory thì stream cho hiệu quả tối ưu tốt nhất và nhanh hơn hẳn so với hai phương pháp trên.
    
    + Kết hợp tất cả phương pháp mang lại kết quả tốt nhất

- Tính đúng đắn:

Độ chính xác đạt 82.96% do đó kết quả khi chạy song song hóa cho lớp Convolution với tối ưu kết hợp cả 3 cách là chính xác vì kết quả này tương đương với khi thực thi trên host.

<a id="4_5"></a>
<h3 style='color:#6ac61f; font-size: 25px'> e. Chọn BlockSize</h3>

Thực hiện chọn BlockSize trên dựa trên kernel cuối cùng (kernel kết hợp tất cả các phương pháp tối ưu). Với BlockSize lần lượt là 4, 16, 32 ta được kết quả:

### BlockSize = 4

![image.png](./image/bs4.png)

### BlockSize = 16

![image-2.png](./image/bs16.png)


### BlockSize = 32

![image-3.png](./image/bs32.png)


### Kết quả

|BlockSize| C1 Time | C3 Time|
|---------|---------|--------|
|4|42.2924|24.1512|
|16|15.9573|12.9168|
|32|18.4023|51.212|


**Nhận xét:** Từ kết quả thực nghiệm có thể thấy, khi blockSize quá nhỏ hoặc quá lớn đều làm tốc độ thực thi của chương trình giảm. Do đó, cần chọn blockSize kích thước phù hợp để chương trình đạt tốc độ tốt nhất.

<a id="5"></a>
<div style=" background-color:#cdf7c1 ; color:#e3395e; padding: 15px; border-radius:5px;text-align: center; border-style: solid; border-color: green; font-size: 35px">5. Tự đánh giá đồ án</div>

<a id="5_1"></a>
<h3 style='color:#6ac61f; font-size: 25px'> a. Những gì đã làm được và chưa làm được</h3>

- Những điều đã làm được:
    + Điều chỉnh code của framework để chạy trên bộ dữ liệu mới
    + Cài đặt song song hóa cho lớp Convolution trong mạng Lenet-5
    + Thực hiện một số cài đặt tối ưu hóa song song:
        + Loop unroll
        + Sử dụng shared memory và constant memory
        + Sử dụng stream
        + Kết hợp các phương pháp tối ưu trên
        + Duyệt qua các blockSize để tìm ra blockSize phù hợp
- Những điều chưa làm được:
    + Giảm số kênh đầu vào

<a id="5_2"></a>
<h3 style='color:#6ac61f; font-size: 25px'> b. Bước tiếp theo và những khó khăn gặp phải và kinh nghiệm giải quyết</h3>

- Nếu có thêm thời gian nhóm em sẽ thực hiện thêm:
    + Giảm số kênh đầu vào
    + Thuật toán nhân ma trận nâng cao
- Khó khăn:
    + Gặp lỗi khi chạy thư viện Eigen trên CUDA
        > Sau khi tìm hiểu nhóm biết được nvcc có thể biên dịch file .cc do đó có thể giữ nguyên file này và import file .cu để chạy.
    + Lỗi khi chạy GridSize với hai chiều với số lượng mẫu đầu vào lớn
        > Thử lại với GridSize ba chiều và kết quả hoạt động
    + Lỗi stream không overlap
        > Sau khi phân tích với NVIDIA Nsight Systems nhóm nhận thấy vùng nhớ của dữ liệu cần copy ở host lên device và ngược lại đang nằm ở Pageable mememory nên chuyển sang pinned memmory.
    

<a id="6"></a>
<div style=" background-color:#cdf7c1 ; color:#e3395e; padding: 15px; border-radius:5px;text-align: center; border-style: solid; border-color: green; font-size: 35px">6. Phân công</div>

|Hoài                             | Nghĩa                                     |
|:------------------------------------------------|:---------------------------------------------------|
| Đọc, tìm hiểu về project                       | Đọc, tìm hiểu về project                          |
| Chỉnh sửa framework để chạy trên tập dữ liệu mới | Chỉnh sửa framework để có kiến trúc Lenet-5       |
| Viết code save, load model                     | Viết thêm lớp Convolution hoạt động trên GPU       |
| Viết hàm kernel song song hóa cơ bản cho Convolution | Viết hàm kernel song song hóa cơ bản cho Convolution |
| Tối ưu song song hóa với unroll, shared memory, constant memory | Tối ưu song song hóa với stream         |
| Viết report                                   | Kết hợp 3 phương pháp tối ưu                     |
|                                               | Viết Makefile, README                            |


**Video demo** : [Đồ án Lập trình song song - Nhóm 13 - HCMUS - 2023-2024](https://www.youtube.com/watch?v=HLLNnBKUuLM)

<a id="7"></a>
<div style=" background-color:#cdf7c1 ; color:#e3395e; padding: 15px; border-radius:5px;text-align: center; border-style: solid; border-color: green; font-size: 35px">7. Tài liệu tham khảo</div>

1. Chat GPT
2. [mini-dnn-cpp](https://github.com/iamhankai/mini-dnn-cpp/tree/master)